In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
!pip install transformers

import sys
sys.path.append('/content/drive/My Drive/colab_files/modules')

import infersent_glove_context_generation as ig

import time
import os
import contextlib
import torch
import nltk
nltk.download('punkt')

from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

In [3]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', return_token_type_ids = True)
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

In [4]:
def get_answer(url, question):
    context = ig.generate_context(url, question)

    encoding = tokenizer.encode_plus(question, context)

    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

    start_scores, end_scores = model(torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))

    ans_tokens = input_ids[torch.argmax(start_scores) : torch.argmax(end_scores)+1]
    answer_tokens = tokenizer.convert_ids_to_tokens(ans_tokens , skip_special_tokens=True)

    print ("\nQuestion ",question)
    print ("\nAnswer Tokens: ", answer_tokens)

    answer_tokens_to_string = tokenizer.convert_tokens_to_string(answer_tokens)
    print("\nAnswer: ", answer_tokens_to_string)
    return answer_tokens_to_string

In [5]:
get_answer('https://en.wikipedia.org/wiki/India', 'What name did the Greeks use to refer to Indians?')

Vocab size : 100000
Nb words kept : 7300/11310 (64.5%)
Speed : 29.6 sentences/s (cpu mode, bsize=128)
Nb words kept : 7/12 (58.3%)
Speed : 9.5 sentences/s (cpu mode, bsize=128)

Question  What name did the Greeks use to refer to Indians?

Answer Tokens:  ['indo', '##i']

Answer:  indoi


'indoi'

**FLASK**

In [6]:
%%capture
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request

app = Flask(__name__, template_folder='drive/My Drive/colab_files/Flask/templates')
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route("/")
def home():
    return render_template('home.html')

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/handle_data', methods=['GET', 'POST'])
def handle_data():
    url = request.form['inputURL']
    question = request.form['inputQuestion']
    answer = get_answer(url, question)
    print('\nAnswer: ', answer)
    return render_template('handle_data.html', result_url=url, result_question=question,
                           result_answer=answer)

app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2020d2f6c129.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [23/Sep/2020 06:04:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2020 06:04:05] "GET /favicon.ico HTTP/1.1" 404 -


Vocab size : 100000
Nb words kept : 7300/11310 (64.5%)
Speed : 29.0 sentences/s (cpu mode, bsize=128)
Nb words kept : 5/9 (55.6%)
Speed : 12.4 sentences/s (cpu mode, bsize=128)

Question  Which are the major rivers in India?

Answer Tokens:  ['gang', '##es', 'and', 'the', 'bra', '##hma', '##put', '##ra']

Answer:  ganges and the brahmaputra

Answer:  ganges and the brahmaputra


127.0.0.1 - - [23/Sep/2020 07:03:52] "POST /handle_data HTTP/1.1" 200 -
